# Load Model

Load persisted model and run experiments

In [122]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

In [123]:
from joblib import load
from pactus import Dataset

from traj_xai.src import ( 
    mdl_segmentation, 
    rdp_segmentation, 
    sliding_window_segmentation, 
    gaussian_perturbation, 
    scaling_perturbation, 
    rotation_perturbation, 
    gan_perturbation,
    run_experiments
)

In [124]:
SEED = 42
# SEED = None

datasets = [
    Dataset.animals,  # 0
    Dataset.geolife,  # 1
    Dataset.hurdat2,  # 2
    Dataset.uci_pen_digits,  # 3
    Dataset.uci_movement_libras,  # 4
    Dataset.uci_characters,  # 5
    Dataset.mnist_stroke  # 6
]

# Load dataset
dataset_idx = 1
dataset = datasets[dataset_idx]()
print(f"Dataset loaded: {len(dataset.trajs)} trajectories ({datasets[dataset_idx].__name__})")

# Split data into train and test subsets
train, test = dataset.split(0.8, random_state=SEED)
print(f"Train set: {len(train.trajs)} trajectories")
print(f"Test set: {len(test.trajs)} trajectories")

Dataset loaded: 9288 trajectories (geolife)
Train set: 7430 trajectories
Test set: 1858 trajectories


In [125]:
model_idx = 0
#             0      1      2       3          4             5
model_str = ["rf", "knn", "lstm", "gru", "transformer", "trajformer"][model_idx]
dataset_str = datasets[dataset_idx].__name__
model_dir = "models"

model_path = f"{model_dir}/{model_str}_{dataset_str}.joblib" 

model = None
if os.path.exists(model_path):
    model = load(model_path)
    print(f"Model loaded: {model_path}")
else:
    print(f"Model file not found: {model_path}")

Model loaded: models/rf_geolife.joblib


In [126]:
# make sure the persisted model works
evaluation = model.evaluate(test)
evaluation.show()

12:12:40 [INFO] Evaluating the random_forest model



General statistics:

Accuracy: 0.876
F1-score: 0.655
Mean precision: 0.694
Mean recall: 0.626

Confusion matrix:

airplane  bike      boat      bus       car       run       subway    taxi      train     walk      precision 
66.67     0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       100.0     
0.0       86.73     0.0       1.64      1.29      0.0       0.85      0.98      0.0       1.04      93.71     
0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       
0.0       4.53      0.0       85.52     13.55     0.0       11.11     27.45     6.06      1.43      77.86     
0.0       0.0       100.0     2.46      81.94     0.0       2.56      8.82      3.03      0.13      84.11     
0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       
0.0       0.32      0.0       2.73      0.0       0.0       76.07     4.9       0.0       0.65      80.91   

In [127]:
import logging

logging.disable(logging.DEBUG)
# logging.disable(logging.NOTSET)

In [128]:
%load_ext autoreload
%autoreload 2

from traj_xai.src import run_experiments, rdp_segmentation, mdl_segmentation, gaussian_perturbation, gan_perturbation

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [130]:
# sliding window currently not working for RF
segment_funcs = [rdp_segmentation, mdl_segmentation]
# time currently doesn't work with GAN and rotation, and scaling somehow creates overflow issues with the RF featurizer
perturbation_funcs = [gaussian_perturbation]

log_idx = 1
# surrogate_params = {'C': 0.5, 'tol': 0.00001, 'max_iter': 500, 'class_weight': 'balanced'}

In [131]:
# without time
run_experiments(test, segment_funcs, perturbation_funcs, model, time=False, 
                log_dataset=dataset_str, log_model=model_str, log_idx=log_idx)

12:13:14 [INFO] Running experiment with rdp_segmentation and gaussian_perturbation
12:13:14 [INFO] Finished explanation process.
12:13:15 [WARNING] Only one class detected, skipping explanation.
12:13:15 [INFO] Trajectory 1: classification unchanged


[INFO] Row saved to logs\geolife_rf_rdp_gaussian_1_results.csv: [0, 'traj_bab3a0e5', 1, 1.0, 'ok']
2025-12-12 12:13:15 - Total RAM: 13.86 GB | Used: 76.2%
[INFO] Row saved to logs\geolife_rf_rdp_gaussian_1_results.csv: [1, 'traj_f0756078', 0, 0.0, 'ok']


12:13:15 [INFO] Finished explanation process.
12:13:15 [WARNING] Only one class detected, skipping explanation.
12:13:15 [INFO] Trajectory 3: classification unchanged
12:13:16 [INFO] Finished explanation process.


[INFO] Row saved to logs\geolife_rf_rdp_gaussian_1_results.csv: [2, 'traj_08b42525', 1, 1.0, 'ok']
[INFO] Row saved to logs\geolife_rf_rdp_gaussian_1_results.csv: [3, 'traj_99feffe0', 0, 0.0, 'ok']
[INFO] Row saved to logs\geolife_rf_rdp_gaussian_1_results.csv: [4, 'traj_d115bc5f', 1, 1.0, 'ok']


12:27:09 [INFO] Finished explanation process.


...

[INFO] Row saved to logs\geolife_rf_mdl_gaussian_1_results.csv: [1854, 'traj_5f411202', 1, 1.0, 'ok']


02:41:06 [WARNING] Only one class detected, skipping explanation.
02:41:06 [INFO] Trajectory 1855: classification unchanged


[INFO] Row saved to logs\geolife_rf_mdl_gaussian_1_results.csv: [1855, 'traj_d2e4354e', 0, 0.0, 'ok']


02:41:07 [INFO] Finished explanation process.
02:41:07 [INFO] Finished explanation process.
02:41:07 [INFO] Results saved to logs\geolife_rf_mdl_gaussian_1_results.csv


[INFO] Row saved to logs\geolife_rf_mdl_gaussian_1_results.csv: [1856, 'traj_c41f4b2b', 1, 1.0, 'ok']
[INFO] Row saved to logs\geolife_rf_mdl_gaussian_1_results.csv: [1857, 'traj_ced2838f', 1, 1.0, 'ok']


In [132]:
# with time
run_experiments(test, segment_funcs, perturbation_funcs, model, time=True, 
                log_dataset=dataset_str, log_model=model_str, log_idx=log_idx)

02:41:08 [INFO] Running experiment with rdp_segmentation and gaussian_perturbation and time
02:41:11 [INFO] Finished explanation process.
02:41:11 [INFO] Finished explanation process.


[INFO] Row saved to logs\geolife_rf_rdp_gaussian_time_1_results.csv: [0, 'traj_bab3a0e5', 1, 1.0, 'ok']
2025-12-13 02:41:11 - Total RAM: 13.86 GB | Used: 75.4%
[INFO] Row saved to logs\geolife_rf_rdp_gaussian_time_1_results.csv: [1, 'traj_f0756078', 1, 1.0, 'ok']


02:41:13 [INFO] Finished explanation process.
02:41:13 [WARNING] Only one class detected, skipping explanation.
02:41:13 [INFO] Trajectory 3: classification unchanged
02:41:13 [INFO] Finished explanation process.


[INFO] Row saved to logs\geolife_rf_rdp_gaussian_time_1_results.csv: [2, 'traj_08b42525', 1, 1.0, 'ok']
[INFO] Row saved to logs\geolife_rf_rdp_gaussian_time_1_results.csv: [3, 'traj_99feffe0', 0, 0.0, 'ok']
[INFO] Row saved to logs\geolife_rf_rdp_gaussian_time_1_results.csv: [4, 'traj_d115bc5f', 1, 1.0, 'ok']


02:55:02 [INFO] Finished explanation process.


...

19:06:31 [INFO] Finished explanation process.


[INFO] Row saved to logs\geolife_rf_rdp_gaussian_time_1_results.csv: [1591, 'traj_8064dd12', 1, 1.0, 'ok']


19:06:31 [INFO] Finished explanation process.


[INFO] Row saved to logs\geolife_rf_rdp_gaussian_time_1_results.csv: [1592, 'traj_48af2b37', 1, 1.0, 'ok']


KeyboardInterrupt: 

In [ ]:
# GAN
run_experiments(test, segment_funcs, [gan_perturbation], model, time=False, 
                log_dataset=dataset_str, log_model=model_str, log_idx=log_idx)

06:29:29 [INFO] Running experiment with rdp_segmentation and gan_perturbation
06:29:34 [WARNING] Only one class detected, skipping explanation.
06:29:34 [INFO] Trajectory 0: classification unchanged


[INFO] Row saved to logs\uci_characters_rf_rdp_gan_1_results.csv: [0, 'traj_0644961d', 0, 0.0, 'ok']
2025-12-12 06:29:34 - Total RAM: 13.86 GB | Used: 61.6%


06:29:39 [WARNING] Only one class detected, skipping explanation.
06:29:39 [INFO] Trajectory 1: classification unchanged


[INFO] Row saved to logs\uci_characters_rf_rdp_gan_1_results.csv: [1, 'traj_10e5b64c', 0, 0.0, 'ok']


06:29:43 [WARNING] Only one class detected, skipping explanation.
06:29:43 [INFO] Trajectory 2: classification unchanged


...

[INFO] Row saved to logs\uci_characters_rf_mdl_gan_1_results.csv: [569, 'traj_c5bc6b96', 0, 0.0, 'ok']


07:47:57 [WARNING] Only one class detected, skipping explanation.
07:47:57 [INFO] Trajectory 570: classification unchanged


[INFO] Row saved to logs\uci_characters_rf_mdl_gan_1_results.csv: [570, 'traj_bf026ef5', 0, 0.0, 'ok']
2025-12-12 07:47:57 - Total RAM: 13.86 GB | Used: 63.5%


07:48:01 [WARNING] Only one class detected, skipping explanation.
07:48:01 [INFO] Trajectory 571: classification unchanged
07:48:01 [INFO] Results saved to logs\uci_characters_rf_mdl_gan_1_results.csv


[INFO] Row saved to logs\uci_characters_rf_mdl_gan_1_results.csv: [571, 'traj_14de252a', 0, 0.0, 'ok']


In [ ]:
# curr = "01"
# segment_idx = int(curr[0])
# perturb_idx = 0
# has_time = int(curr[1])

def get_results(segment_idx, perturb_idx, has_time, gan_override=False):
    segment_str = str(segment_funcs[segment_idx].__name__).rsplit("_", 1)[0]
    perturb_str = str(perturbation_funcs[perturb_idx].__name__).rsplit("_", 1)[0]
    if gan_override:
        perturb_str = str(gan_perturbation.__name__).rsplit("_", 1)[0]

    file_path = os.path.join(
        "logs",
        f"{dataset_str}_{model_str}_{segment_str}_{perturb_str}_{'time_' if has_time else ''}{f'{log_idx}_' if log_idx else ''}results.csv",
    )

    count = 0
    total_change = 0
    avg_precision = 0
    with open(file_path, 'r') as f:
        for i, line in enumerate(f):
            if i > 0 and len(line) > 0:
                count += 1
                _, _, change, precision, status = line.strip().split(',')
                if status == 'ok':
                    total_change += int(change)
                    avg_precision += float(precision)
    recall = total_change / count
    avg_precision = (avg_precision / total_change) if total_change != 0 else 0.0
    h_mean = ((2 * avg_precision * recall) / (avg_precision + recall)) if total_change != 0 else 0.0
    # print(f"{file_path}\nRecall: {recall:.3f}\nPrecision: {avg_precision:.3f}\nF1 Score: {h_mean:.3f}")
    print(f"{recall:.3f}\t{avg_precision:.3f}\t{h_mean:.3f}")

In [133]:
# Gaussian, with/without time
for t in range(2):
    for seg in range(2):
        get_results(seg, 0, t)

0.779	1.000	0.876
0.551	1.000	0.710
0.858	1.000	0.924


FileNotFoundError: [Errno 2] No such file or directory: 'logs\\geolife_rf_mdl_gaussian_time_1_results.csv'

In [ ]:
# GAN
for seg in range(2):
    get_results(seg, 0, 0, True)

0.000	0.000	0.000
0.000	0.000	0.000


side note: the predicted labels passed to the surrogate model is decoded into strings for some reason, which seems useless because in the surrogate model function, if the Y labels are strings it encodes them again using a LabelEncoder

side note: sliding window segmentation was implemented incorrecly, the `step` parameter doesn't make sense to specify, since it should always be the window size determined by `percentage`, unless the idea is to skip points between segements, but then the parameter should be `skip` be added on top of the default step size...